# Getting the data

In [1]:
import requests
import json

First find the article ID for the Harry Potter page.

In [2]:
r = requests.get('http://harrypotter.wikia.com/api/v1/Articles/Top')
hp_id = [it['id'] for it in json.loads(r.content)['items'] if it['title'] == 'Harry Potter'][0]


Get the content of the page as a JSON file and dump it to file.

In [3]:
r = requests.get('http://harrypotter.wikia.com/api/v1/Articles/AsSimpleJson', params={'id': hp_id})
json.dump(r.text, open('HarryPotterWikia1.json', 'w'))

## Pulling out the text

In [4]:
cont = json.loads(r.text)

In [5]:
with open('HarryPotterWikia.txt', 'w') as f:
    for section in cont['sections']:
        #print(section['title'])
        #f.write(section['title'].encode('utf8')+'\n')
        f.write(section['title']+'\n')
        for unit in section['content']:
            if unit['type'] == 'paragraph':
                f.write(unit['text']+'\n')

# Playing with NLP on the text

In [6]:
import textacy

In [118]:
from itertools import tee

def make_list(generator):
    entries = []
    for a in generator:
        entries.append(a)
    return entries

def semi_structured(doc):
    statements = textacy.extract.semistructured_statements(doc, 'Harry', cue='be')
    for i, s in enumerate(statements):
        #if i < 10:
            the_rest = textacy.Doc(s[2].text, lang='en')
            names = make_list(textacy.extract.named_entities(the_rest))
            nouns = textacy.extract.noun_chunks(the_rest)
            #triplets = textacy.extract.subject_verb_object_triples(the_rest)
            ngram = make_list(textacy.extract.ngrams(the_rest,5))
            #print(is_n_empty(names)," ",is_n_empty(ngram) )    

            if len(names)*len(ngram)> 0:
                print(" -- My sentence")
                print(s)
                print(" --- Other names")
                for i_name in names:
                    print(i_name)
            #print(" --- Nouns")
            #for i_noun in nouns:
            #    print(i_noun)
                print(" --- ngram")
                for i_gram in ngram:
                    print(i_gram)
        #sents_parsed = dependency_parser.parse_sents(sentences=s[2].text)        
#        print(sents_parsed)

In [247]:
import nltk

def extract_pattern(doc):
    statements = textacy.extract.semistructured_statements(doc, 'Harry', cue='be')
    for i, s in enumerate(statements):
        if i < 20:

            to_tokenize = s[0].text+" "+s[1].text+" "+s[2].text
            the_rest = textacy.Doc(to_tokenize, lang='en')  
            main_verbs = textacy.spacy_utils.get_main_verbs_of_sent(the_rest)
            for verb in main_verbs:
                span = textacy.spacy_utils.get_span_for_verb_auxiliaries(verb)
                sub_list = []
                ob_list = []
                complex_verb = the_rest[span[0]].text
                for a in range(span[0]+1,span[1]+1):
                    complex_verb +=" "+the_rest[span[1]].text
                ##### -----
                subjects = textacy.spacy_utils.get_subjects_of_verb(verb)
                objects = textacy.spacy_utils.get_objects_of_verb(verb)
                if len(subjects) >0 and len(objects) > 0:
                    print(" -- My sentence")
                    print(s)
                    print(subjects,complex_verb,objects)
                
            

In [248]:
text = textacy.preprocess.transliterate_unicode(open('HarryPotterWikia.txt').read())
doc = textacy.Doc(text, lang='en')
#semi_structured(doc)
extract_pattern(doc)
#print(doc[1])
#triple(doc)
#statements = textacy.extract.semistructured_statements(doc, 'Harry')

 -- My sentence
(Harry, was, under increasing pressure to show that he was not just a famous name)
1
[he] was not [name]
 -- My sentence
(Harry, was, to be replaced amusing, and so reverted to teasing Harry about having to stay at Hogwarts for the holidays)
1
[Harry] was [replaced, reverted]
 -- My sentence
(Harry, was, Slytherin's heir)
1
[Harry] was [heir]
 -- My sentence
(Harry, were, furious with Hermione and they stopped speaking to her)
1
[they] stopped [speaking]
 -- My sentence
(Harry, was, sceptical until Black and Lupin forced Pettigrew back into his human form)
2
[Black, Lupin] forced [Pettigrew]
 -- My sentence
(Harry, was, not particularly concerned about and something he did not inform anyone of)
1
[he] did inform inform [anyone]
 -- My sentence
(Harry, was, free of the spell that petrified him)
1
[that] petrified [him]
 -- My sentence
(Harry, was, still completely grief-stricken over the death of his beloved owl)
1
[Harry] was [grief]
 -- My sentence
(Harry, were, the on

In [10]:
#sents_parseobjs = [obj for sent in sents_parsed for obj in sent]
#sents_parseobjs[0].tree()

NameError: name 'sents_parsed' is not defined

In [ ]:
sovs = textacy.extract.subject_verb_object_triples(doc)

In [ ]:
for i, sov in enumerate(sovs):
    if i < 20:
        print(sov)

In [ ]:
textacy.keyterms.key_terms_from_semantic_network(doc)